In [91]:
import random
from pymongo import MongoClient
import pandas as pd
import folium

In [57]:
#Taking the mean of a random set of coordinates all in San Francisco

lats= [37.7084 + random.random()*(37.8107-37.7084) for n in range(10000)]
longs= [-122.3633 - random.random()*( 122.5024 -122.3633) for n in range(10000)]

mean_lats = sum(lats)/len(lats)
mean_longs = sum(longs)/len(longs)


print(mean_lats)
print(mean_longs)

37.75985202991383
-122.43324458478466


In [58]:
point=dict()
point["type"]="Point"
point["coordinates"]=[mean_longs,mean_lats]



In [59]:
point

{'type': 'Point', 'coordinates': [-122.43324458478466, 37.75985202991383]}

In [78]:
dbName = "project_mongo"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()



query = {"$and":
          [{ "geopoint" :
           { "$near" :
           { "$geometry" : point,
               "$maxDistance" : 2500} } },
  ]} 

projection = {"_id":0,"name":1,"formatted_address":1, "latitude":1, "longitude":1}


cur_starbucks = db.starbucks.find_one(query, projection)
cur_starbucks

{'formatted_address': '4094 18th St, San Francisco, CA 94114, United States',
 'name': 'Starbucks',
 'latitude': 37.761019,
 'longitude': -122.434666}

In [79]:
cur_schools = db.schools.find_one(query, projection)
cur_schools

{'formatted_address': '150 Oak St, San Francisco, CA 94102, United States',
 'name': 'International High School',
 'latitude': 37.7753816,
 'longitude': -122.4216371}

In [80]:
cur_pepe = db.pepe_hairdresser.find_one(query, projection)
cur_pepe

{'formatted_address': '536 Castro St, San Francisco, CA 94114, United States',
 'name': "Mudpuppy's",
 'latitude': 37.7602933,
 'longitude': -122.4351867}

In [81]:
cur_vegan = db.vegan.find_one(query, projection)
cur_vegan

{'formatted_address': '762 Valencia St, San Francisco, CA 94110, United States',
 'name': 'Cha-Ya',
 'latitude': 37.760768,
 'longitude': -122.421698}

In [105]:
m = folium.Map(
    location=[point["coordinates"][1], point["coordinates"][0]],
    #tiles='Stamen Toner',
    zoom_start=13
)
    
folium.Marker(
    location=[cur_starbucks["latitude"], cur_starbucks["longitude"]],
    popup='Starbucks',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[cur_schools["latitude"], cur_schools["longitude"]],
    popup='Nearby School',
    icon=folium.Icon(color='gray')
).add_to(m)

folium.Marker(
    location=[cur_pepe["latitude"], cur_pepe["longitude"]],
    popup="Pepe's hairdresser",
    icon=folium.Icon(color='red')
).add_to(m)

folium.Marker(
    location=[cur_vegan["latitude"], cur_vegan["longitude"]],
    popup="CEO's vegan restaurant",
    icon=folium.Icon(color='green')
).add_to(m)

folium.Circle(
    radius=2000,
    location=[point["coordinates"][1], point["coordinates"][0]],
    popup='radius',
    color='gray',
    fill=True,
).add_to(m)

m